In [1]:
from mylibrary.secrets import app_id, app_key, orig_name, dest_name, orig_pc, dest_pc
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session


/Users/robinlinacre/anaconda/lib/python3.6/site-packages/sqlalchemy/dialects/postgresql/base.py:2526: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))


In [2]:
# conn.rollback()

# sql = """
# drop table if exists tt_h.stations_journeytime;

# create table tt_h.stations_journeytime as 
# select nlc, icscode, station_name from tt_h.all_stations where icscode_status= 'ok';
# """
# cur = conn.cursor()
# cur.execute(sql)
# conn.commit()

# sql = """
# ALTER TABLE tt_h.stations_journeytime
# ADD id SERIAL,
# ADD tfl_request text, 
# ADD tfl_response json,
# ADD querydict json,
# ADD tfl_message text,
# ADD tfl_dest text;

# ALTER TABLE tt_h.stations_journeytime ADD PRIMARY KEY (id);

# """
# cur = conn.cursor()
# cur.execute(sql)
# conn.commit()

In [3]:
# Dump all the json to the database first.  Then process it later
# Iterate through adding icscode
import requests
import pandas as pd
import json

from mylibrary.tfl_helpers import get_journeyplanner_raw_content, status_of_tfl_response_journey, get_journeyplanner_results

In [4]:
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

def get_attempt(pc, times, dates):

    attempt = {
        "to": pc,
        "id": app_id,
        "key": app_key,
        "time": times[0],
        "date": dates[0],
        "nationalsearch": True}
                      
    param_dicts = []
    param_dicts.append(attempt)

    d1 = attempt.copy()
    d1["time"] =times[1]
    d1["date"] = dates[1]
    param_dicts.append(d1)

    d2 = attempt.copy()
    d2["time"] =times[2]
    d2["date"] = dates[2]
    param_dicts.append(d2)

    return param_dicts

In [5]:
# Multithread 

import threading
num_threads = 6

from sqlalchemy import or_

from sqlalchemy.orm.session import make_transient

Stations2 = Automapped_Base.classes.stations_journeytime


# Needs only the nlcs which haven't already been done

sql = """
select distinct nlc from tt_h.stations_journeytime  
where nlc not in (select nlc from tt_h.stations_journeytime where tfl_message ='ok')
"""

nlcs = list(pd.read_sql(sql,conn)["nlc"])
nlcs = [int(i) for i in nlcs]


In [ ]:

# nlcs  = session.query(Stations2.nlc).filter(or_(Stations2.tfl_message == None, Stations2.tfl_message != 'ok')).all()
counter = 1
from sqlalchemy import create_engine

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, MetaData

def new_thread():
    global counter
    
    engine = create_engine('postgresql://robinlinacre:@localhost:5432/postgres') 
    m = MetaData(schema='tt_h')
    Automapped_Base = automap_base(bind=engine,metadata=m)
    Automapped_Base.prepare(engine, reflect=True)
    session = Session(engine)
    Stations = Automapped_Base.classes.stations_journeytime

    
    while len(nlcs) > 0:
        nlc = nlcs.pop()
        station = session.query(Stations).filter(Stations.nlc == nlc).first()
        station_id = station.id

        attempts = []
        for dest in ["SW1H9AJ", "E145HP"]:
            attempts.append(get_attempt(dest, ["0800","0820", "0840"], ["20170817","20170906","20170908"]))
            attempts.append(get_attempt(dest, ["0820","0840", "0900"], ["20170817","20170906","20170908"]))
            attempts.append(get_attempt(dest, ["0740","0800", "0820"], ["20170817","20170906","20170908"]))
    
        for param_dicts in attempts:
            
            for d in param_dicts:
                d["from"] = station.icscode
            

            journey_object = get_journeyplanner_results(param_dicts)
            
            _del = station.station_name # This seems to force retreival from the db
            
            session.expunge(station)
            make_transient(station)
            station.id = None

            station.tfl_request = journey_object["request_url"]
            station.tfl_response = journey_object["tfl_response"]
            station.querydict = journey_object["query_dict"]
            station.tfl_message = journey_object["tfl_message"]
            station.tfl_dest = param_dicts[0]["to"]

            session.add(station)
            
            session.commit()
            
            counter +=1
            if counter % 10 == 0:
                print(counter)
        
        original = session.query(Stations).filter(Stations.id == station_id)
        original.delete()
        session.commit()
        
        
 
my_threads = []

for i in range(num_threads):
    t = threading.Thread(target=new_thread)
    my_threads.append(t)
    t.start() #start this thread

for x in my_threads:
    x.join()

/Users/robinlinacre/anaconda/lib/python3.6/site-packages/sqlalchemy/dialects/postgresql/base.py:2526: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
Traceback (most recent call last):
  File "/Users/robinlinacre/Documents/python_projects/house_tt/mylibrary/tfl_helpers.py", line 30, in status_of_tfl_response_journey
    parsed_response = json.loads(content)
  File "/Users/robinlinacre/anaconda/lib/python3.6/json/__init__.py", line 354, in loads
    return _default_decoder.decode(s)
  File "/Users/robinlinacre/anaconda/lib/python3.6/json/decoder.py", line 339, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/Users/robinlinacre/anaconda/lib/python3.6/json/decoder.py", line 357, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370


In [ ]:
# How many stations are there in total
sql = """
select count(*) from (select distinct(icscode) from tt_h.stations_journeytime_cw) as a

"""
pd.read_sql(sql, conn)

In [ ]:
# How many stations are there that we have an 'ok' for?
sql = """
select count(*) from (select distinct(icscode) from tt_h.stations_journeytime_cw where tfl_message = 'ok') as a

"""
pd.read_sql(sql, conn)

In [ ]:
get_attempt("SW1H9AJ", ["0800","0820", "0840"], ["20170817","20170906","20170908"])